In [8]:
import pandas as pd
import numpy as np
import unicodedata
from datetime import datetime

# --- helpers ---
def strip_accents(s):
    if pd.isna(s): return ""
    return ''.join(c for c in unicodedata.normalize('NFD', str(s))
                   if not unicodedata.combining(c))

def parse_date(x):
    return pd.to_datetime(x, errors="coerce", dayfirst=True)

# --- XIRR ---
def xnpv(rate, cashflows):
    if not cashflows: return np.nan
    cfs = sorted(cashflows, key=lambda t: t[0])
    t0 = cfs[0][0]
    return sum(cf / ((1 + rate) ** ((d - t0).days / 365)) for d, cf in cfs)

def dxnpv(rate, cashflows):
    cfs = sorted(cashflows, key=lambda t: t[0])
    t0 = cfs[0][0]
    return sum(-(cf * (d - t0).days / 365) / ((1 + rate) ** ((d - t0).days / 365 + 1))
               for d, cf in cfs)

def xirr(cashflows, guess=0.1, tol=1e-7, max_iter=100):
    r = guess
    for _ in range(max_iter):
        f, df = xnpv(r, cashflows), dxnpv(r, cashflows)
        if abs(f) < tol: return r
        if abs(df) < 1e-12: r += 1e-3; continue
        r_new = r - f/df
        if abs(r_new - r) < tol: return r_new
        r = r_new
    return np.nan

# --- cleaning ---
def clean_movements(df):
    df = df.copy()
    df["operation_date"] = df["operation_date"].apply(parse_date)
    df["desc_norm"] = df["description"].map(strip_accents).str.upper()
    df = df[df["desc_norm"].isin(["DEPOSITO","RETIRO"])]

    df["cash_flow"] = df["movement_import"].astype(float)
    df.loc[df["desc_norm"]=="DEPOSITO","cash_flow"] *= -1
    return df[["contract","operation_date","description","cash_flow"]]

def clean_balances(df):
    df = df.copy()
    df["operation_date"] = df["balance_date"].apply(parse_date)  # <- fix here
    out = df.groupby(["contract","operation_date"],as_index=False)["value_pos_mdo"].sum()
    out = out.rename(columns={"value_pos_mdo":"portfolio_value"})
    return out

# --- build flows & compute MWRR ---
def compute_mwrr(movements, balances):
    movs = clean_movements(movements)
    bals = clean_balances(balances)

    results = {}
    for c in set(movs["contract"]) | set(bals["contract"]):
        flows = []
        for _, r in movs[movs["contract"]==c].iterrows():
            flows.append((r["operation_date"], r["cash_flow"]))
        last = bals[bals["contract"]==c].sort_values("operation_date").tail(1)
        if not last.empty:
            flows.append((last["operation_date"].iloc[0], last["portfolio_value"].iloc[0]))
        if flows and min(v for _,v in flows) < 0 and max(v for _,v in flows) > 0:
            results[c] = xirr(flows)*100
        else:
            results[c] = np.nan
    return results


In [11]:
xls = pd.ExcelFile("data_actividad.xlsx")
df_mov = pd.read_excel(xls, sheet_name="movements")
df_bal = pd.read_excel(xls, sheet_name="balances")

mwrrs = compute_mwrr(df_mov, df_bal)

for c, r in mwrrs.items():
    if pd.notna(r):
        print(f"Cliente {c}: {r:.2f}%")
    else:
        print(f"Cliente {c}: no se pudo calcular")
